In [1]:
import tensorflow
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
import tensorflow as tf

In [33]:
import pandas as pd
import sklearn
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import metrics
import nltk
import string
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
import re
from nltk.stem.porter import *

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Quinton/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Quinton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv('nuforc_reports.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112095 entries, 0 to 112094
Data columns (total 12 columns):
summary           112059 non-null object
city              111851 non-null object
state             103722 non-null object
date_time         109726 non-null object
shape             108580 non-null object
duration          108382 non-null object
stats             112042 non-null object
report_link       112095 non-null object
text              112041 non-null object
posted            109726 non-null object
city_latitude     89359 non-null float64
city_longitude    89359 non-null float64
dtypes: float64(2), object(10)
memory usage: 10.3+ MB


In [5]:
df.isnull().sum()

summary              36
city                244
state              8373
date_time          2369
shape              3515
duration           3713
stats                53
report_link           0
text                 54
posted             2369
city_latitude     22736
city_longitude    22736
dtype: int64

# Tokenizing and Vectorizing Text

In [6]:
df[df['text'].isnull()]

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
17151,Glowing fireball object crashed at Maverick's...,El Granada,CA,1995-01-01T02:00:00,fireball,45 seconds,NaN,http://www.nuforc.org/webreports/025/S25864.html,NaN,2002-11-04T00:00:00,37.502700,-122.469400
22560,Objects flickering red and green seemingly hov...,Toronto,ON,2003-08-16T22:30:00,unknown,1 hour 30. minutes +,NaN,http://www.nuforc.org/webreports/030/S30727.html,NaN,2003-08-28T00:00:00,43.675886,-79.414188
24968,Explosion in the sky,New Effington,SD,2004-02-14T20:17:00,unknown,2 seconds,NaN,http://www.nuforc.org/webreports/035/S35101.html,NaN,2004-03-02T00:00:00,45.862000,-96.948100
25235,chevron shaped craft hovering near our home,Monkland,ON,2004-02-19T19:30:00,chevron,30 Minutes,NaN,http://www.nuforc.org/webreports/035/S35193.html,NaN,2004-03-02T00:00:00,45.200100,-74.866000
25984,Bright greenish / white light that flew in a s...,Gatineau,QC,2003-11-26T21:41:00,light,3 Seconds,NaN,http://www.nuforc.org/webreports/033/S33439.html,NaN,2003-12-09T00:00:00,45.484218,-75.670763
27151,"three nights, in one week, glowing green and r...",Santa Fe,NM,2004-05-05T20:30:00,oval,1/2 hour,NaN,http://www.nuforc.org/webreports/036/S36882.html,NaN,2004-06-04T00:00:00,35.661083,-105.953907
27197,ufo at sunset,Seattle,WA,2004-05-12T19:15:00,light,15-20 sec,NaN,http://www.nuforc.org/webreports/036/S36880.html,NaN,2004-06-04T00:00:00,47.609046,-122.326028
27201,oval shape bright as a full moon,Jacksonville,FL,2004-05-13T00:20:00,oval,2 seconds,NaN,http://www.nuforc.org/webreports/036/S36883.html,NaN,2004-06-04T00:00:00,30.266905,-81.616373
27213,lots of UFOs,Perth (Western Australia),NaN,2004-05-13T23:00:00,cigar,50,NaN,http://www.nuforc.org/webreports/036/S36881.html,NaN,2004-06-04T00:00:00,NaN,NaN
27699,"Two separate ""fireball"" incidents in very dist...",Bullhead City,AZ,1987-06-15T18:30:00,fireball,?? 10min ??,NaN,http://www.nuforc.org/webreports/036/S36103.html,NaN,2004-04-27T00:00:00,35.084078,-114.562823


In [7]:
#replace missing text with summary
df['text']= df['text'].fillna(value = df['summary'])

In [8]:
#drop rows where there's no summary or text
reports = df.dropna(axis=0, how='all', subset=['text','summary'])

In [9]:
reports.isnull().sum()

summary              35
city                244
state              8373
date_time          2369
shape              3515
duration           3713
stats                53
report_link           0
text                  0
posted             2369
city_latitude     22736
city_longitude    22736
dtype: int64

In [10]:
#replace missing text with summary
reports['text'].fillna(value = reports['summary'], inplace=True)

/Users/Quinton/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [11]:
reports['text'].isnull().sum()

0

In [12]:
reports.isnull().sum()

summary              35
city                244
state              8373
date_time          2369
shape              3515
duration           3713
stats                53
report_link           0
text                  0
posted             2369
city_latitude     22736
city_longitude    22736
dtype: int64

In [13]:
def remove_punctuation (text):
  
  new_text = "".join([char for char in text if char not in string.punctuation])
  return new_text

reports['clean_text'] = reports['text'].astype(str).apply(lambda x:remove_punctuation(x))


/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
def tokenize (text):
  tokens = re.split('\W+', text)
  return tokens [1:]

reports['tokenized_text'] = reports['clean_text'].apply(lambda x: tokenize(x))


/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(tokenized_text):
  text = [word for word in tokenized_text if word not in stopword]
  return text

reports['nostop_text']= reports['tokenized_text'].apply(lambda x:remove_stopwords(x))


/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Creating new dataframe with vectors

In [76]:
reports['date_time'] = pd.to_datetime(reports['date_time'])

/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [104]:
# Separate the year, month, day, hour, and minute from our datetime data.

reports['year'] = reports['date_time'].map(lambda x: x.year)
reports['month'] = reports['date_time'].map(lambda x: x.month)
reports['day'] = reports['date_time'].map(lambda x: x.day)
reports['hour'] = reports['date_time'].map(lambda x: x.hour)
reports['minute'] = reports['date_time'].map(lambda x: x.minute)

/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [107]:
# Parsing duration for parsable entries
def extract_duration(text):
    if(type(text) != 'str'):
        text = str(text)
    if(text == ""):
        return -1
    elif(not any(char.isdigit() for char in text)):
        return -1
    else:
        # first numeral value
        start_idx = None
        end_idx = None
        units_idx = None
        for i in range(len(text)):
            if(text[i].isdigit()):
                start_idx = i
#                 print("start_idx="+str(start_idx))
                break
#         print("end of loop start_idx:"+str(start_idx))
        for i in range(start_idx,len(text)):
            if(not text[i].isdigit()):
                end_idx = i
#                 print("text["+str(i)+"]="+str(text[i]))
#                 print("end_idx="+str(end_idx))
                break
        if(end_idx == None):
            return -1
        duration_str = text[start_idx:end_idx]
        print("duration_str="+str(duration_str))
        duration = int(duration_str)
        for i in range(end_idx,end_idx+min(5,len(text)-end_idx)):
            if(text[i] == "m" or text[i] == "M" or text[i] == "s" or text[i] == "S" or text[i] == "h" or text[i] == "H"):
                units_idx = i
                break
        if(units_idx == None):
            return -1
        if(text[units_idx] == "h" or text[units_idx] == "H"):
            return duration*3600
        if(text[units_idx] == "m" or text[units_idx] == "M"):
            return duration*60
        if(text[units_idx] == "s" or text[units_idx] == "S"):
            return duration

reports['duration_parsed'] = reports['duration'].apply(lambda x:extract_duration(x))

duration_str=15
duration_str=5
duration_str=90
duration_str=5
duration_str=15
duration_str=45
duration_str=3
duration_str=1
duration_str=5
duration_str=2
duration_str=30
duration_str=30
duration_str=30
duration_str=8
duration_str=1
duration_str=20
duration_str=5
duration_str=5
duration_str=30
duration_str=3
duration_str=5
duration_str=10
duration_str=5
duration_str=30
duration_str=5
duration_str=10
duration_str=10
duration_str=2
duration_str=20
duration_str=1
duration_str=1
duration_str=20
duration_str=4
duration_str=10
duration_str=20
duration_str=4
duration_str=4
duration_str=5
duration_str=30
duration_str=5
duration_str=5
duration_str=1
duration_str=1
duration_str=30
duration_str=5
duration_str=3
duration_str=1
duration_str=5
duration_str=15
duration_str=10
duration_str=10
duration_str=12
duration_str=2
duration_str=3
duration_str=5
duration_str=5
duration_str=30
duration_str=30
duration_str=15
duration_str=3
duration_str=3
duration_str=45
duration_str=15
duration_str=10
duration_st

duration_str=5
duration_str=10
duration_str=3
duration_str=25
duration_str=90
duration_str=2
duration_str=20
duration_str=1
duration_str=10
duration_str=30
duration_str=5
duration_str=5
duration_str=1
duration_str=4
duration_str=3
duration_str=45
duration_str=45
duration_str=10
duration_str=20
duration_str=1
duration_str=10
duration_str=5
duration_str=90
duration_str=5
duration_str=1
duration_str=5
duration_str=30
duration_str=5
duration_str=7
duration_str=7
duration_str=2
duration_str=30
duration_str=45
duration_str=10
duration_str=1
duration_str=1
duration_str=15
duration_str=3
duration_str=20
duration_str=0
duration_str=5
duration_str=20
duration_str=2
duration_str=5
duration_str=20
duration_str=15
duration_str=15
duration_str=5
duration_str=12
duration_str=2
duration_str=10
duration_str=5
duration_str=10
duration_str=5
duration_str=25
duration_str=30
duration_str=30
duration_str=10
duration_str=1
duration_str=5
duration_str=20
duration_str=5
duration_str=10
duration_str=58
duration

duration_str=5
duration_str=2
duration_str=4
duration_str=4
duration_str=4
duration_str=30
duration_str=2
duration_str=6
duration_str=4
duration_str=3
duration_str=45
duration_str=4
duration_str=6
duration_str=1
duration_str=40
duration_str=1
duration_str=30
duration_str=55
duration_str=2
duration_str=10
duration_str=3
duration_str=7
duration_str=5
duration_str=15
duration_str=3
duration_str=4
duration_str=4
duration_str=23
duration_str=30
duration_str=20
duration_str=10
duration_str=5
duration_str=3
duration_str=2
duration_str=5
duration_str=20
duration_str=40
duration_str=5
duration_str=5
duration_str=4
duration_str=45
duration_str=20
duration_str=90
duration_str=14
duration_str=20
duration_str=65
duration_str=45
duration_str=35
duration_str=35
duration_str=30
duration_str=2
duration_str=30
duration_str=10
duration_str=5
duration_str=15
duration_str=20
duration_str=70
duration_str=2
duration_str=3
duration_str=10
duration_str=7
duration_str=3
duration_str=3
duration_str=10
duration_s

duration_str=10
duration_str=2
duration_str=8
duration_str=10
duration_str=5
duration_str=5
duration_str=20
duration_str=40
duration_str=1
duration_str=2
duration_str=5
duration_str=08
duration_str=35
duration_str=5
duration_str=30
duration_str=30
duration_str=3
duration_str=5
duration_str=2
duration_str=20
duration_str=10
duration_str=2
duration_str=20
duration_str=3
duration_str=30
duration_str=5
duration_str=3
duration_str=19
duration_str=3
duration_str=1
duration_str=3
duration_str=4
duration_str=15
duration_str=5
duration_str=3
duration_str=5
duration_str=5
duration_str=5
duration_str=25
duration_str=2
duration_str=2
duration_str=2
duration_str=40
duration_str=30
duration_str=1
duration_str=45
duration_str=3
duration_str=15
duration_str=40
duration_str=10
duration_str=10
duration_str=3
duration_str=2
duration_str=5
duration_str=2
duration_str=3
duration_str=30
duration_str=10
duration_str=6
duration_str=1
duration_str=5
duration_str=5
duration_str=30
duration_str=3
duration_str=15

duration_str=3
duration_str=30
duration_str=90
duration_str=2
duration_str=5
duration_str=3
duration_str=2
duration_str=5
duration_str=10
duration_str=1
duration_str=2
duration_str=1
duration_str=15
duration_str=3
duration_str=20
duration_str=30
duration_str=25
duration_str=2
duration_str=2
duration_str=11
duration_str=30
duration_str=20
duration_str=5
duration_str=10
duration_str=5
duration_str=10
duration_str=10
duration_str=3
duration_str=30
duration_str=2
duration_str=10
duration_str=15
duration_str=3
duration_str=30
duration_str=1
duration_str=5
duration_str=5
duration_str=1
duration_str=30
duration_str=30
duration_str=12
duration_str=15
duration_str=15
duration_str=30
duration_str=45
duration_str=4
duration_str=15
duration_str=5
duration_str=2
duration_str=1
duration_str=3
duration_str=1
duration_str=30
duration_str=45
duration_str=2
duration_str=15
duration_str=5
duration_str=1
duration_str=5
duration_str=2
duration_str=5
duration_str=2
duration_str=5
duration_str=5
duration_str

...

duration_str=2
duration_str=5
duration_str=30
duration_str=3
duration_str=5
duration_str=15
duration_str=1
duration_str=1
duration_str=3
duration_str=5
duration_str=1
duration_str=45
duration_str=15
duration_str=5
duration_str=5
duration_str=30
duration_str=15
duration_str=3
duration_str=2
duration_str=2
duration_str=2
duration_str=5
duration_str=4
duration_str=15
duration_str=2
duration_str=5
duration_str=10
duration_str=5
duration_str=1
duration_str=2
duration_str=2
duration_str=2
duration_str=3
duration_str=5
duration_str=2
duration_str=5
duration_str=3
duration_str=2
duration_str=5
duration_str=1
duration_str=2
duration_str=2
duration_str=3
duration_str=5
duration_str=10
duration_str=30
duration_str=4
duration_str=2
duration_str=10
duration_str=2
duration_str=10
duration_str=5
duration_str=2
duration_str=30
duration_str=7
duration_str=5
duration_str=40
duration_str=3
duration_str=1
duration_str=10
duration_str=15
duration_str=30
duration_str=30
duration_str=1
duration_str=5
duratio

duration_str=3
duration_str=2
duration_str=20
duration_str=10
duration_str=30
duration_str=5
duration_str=7
duration_str=2
duration_str=2
duration_str=40
duration_str=1
duration_str=2
duration_str=30
duration_str=10
duration_str=1
duration_str=3
duration_str=8
duration_str=30
duration_str=5
duration_str=1
duration_str=5
duration_str=20
duration_str=4
duration_str=5
duration_str=2
duration_str=2
duration_str=10
duration_str=5
duration_str=10
duration_str=1
duration_str=5
duration_str=3
duration_str=20
duration_str=1
duration_str=20
duration_str=20
duration_str=3
duration_str=10
duration_str=15
duration_str=3
duration_str=1
duration_str=1
duration_str=30
duration_str=5
duration_str=3
duration_str=5
duration_str=3
duration_str=3
duration_str=2
duration_str=5
duration_str=2
duration_str=10
duration_str=10
duration_str=10
duration_str=20
duration_str=30
duration_str=2
duration_str=3
duration_str=1
duration_str=20
duration_str=30
duration_str=10
duration_str=15
duration_str=1
duration_str=15

duration_str=35
duration_str=2
duration_str=1
duration_str=30
duration_str=30
duration_str=7
duration_str=2
duration_str=3
duration_str=1
duration_str=3
duration_str=30
duration_str=1
duration_str=15
duration_str=1
duration_str=2
duration_str=5
duration_str=5
duration_str=2
duration_str=20
duration_str=10
duration_str=10
duration_str=30
duration_str=7
duration_str=15
duration_str=2
duration_str=60
duration_str=4
duration_str=5
duration_str=1
duration_str=2
duration_str=3
duration_str=1
duration_str=5
duration_str=1
duration_str=3
duration_str=3
duration_str=2
duration_str=5
duration_str=2
duration_str=1
duration_str=1
duration_str=2
duration_str=30
duration_str=30
duration_str=10
duration_str=10
duration_str=10
duration_str=1
duration_str=3
duration_str=90
duration_str=15
duration_str=40
duration_str=3
duration_str=5
duration_str=5
duration_str=10
duration_str=3
duration_str=5
duration_str=5
duration_str=10
duration_str=15
duration_str=1
duration_str=3
duration_str=2
duration_str=3
dur

duration_str=20
duration_str=2
duration_str=1
duration_str=5
duration_str=6
duration_str=5
duration_str=1
duration_str=30
duration_str=20
duration_str=1
duration_str=5
duration_str=5
duration_str=15
duration_str=5
duration_str=2
duration_str=3
duration_str=1
duration_str=3
duration_str=2
duration_str=2
duration_str=3
duration_str=30
duration_str=5
duration_str=5
duration_str=5
duration_str=2
duration_str=4
duration_str=90
duration_str=1
duration_str=5
duration_str=2
duration_str=9
duration_str=6
duration_str=10
duration_str=1
duration_str=10
duration_str=5
duration_str=10
duration_str=9
duration_str=4
duration_str=30
duration_str=15
duration_str=2
duration_str=30
duration_str=30
duration_str=21
duration_str=2
duration_str=2
duration_str=3
duration_str=1
duration_str=15
duration_str=44
duration_str=30
duration_str=3
duration_str=3
duration_str=10
duration_str=5
duration_str=4
duration_str=11
duration_str=15
duration_str=1
duration_str=30
duration_str=10
duration_str=10
duration_str=15
d

/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [109]:
# Separate location data into regions
reports['citystate'] = reports['city'] + "!" + reports['state']

def region_separate(citystate):
    pacific = ['WA','OR','CA']
    rockies = ['NV','ID','MT','WY','UT','CO']
    southwest = ['AZ','NM','TX','OK']
    midwest = ['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH']
    southeast = ['AR','LA','TN','MS','KY','AL','FL','GA','SC','NC','VA','WV','MD','DE']
    northeast = ['PA','NY','NJ','CT','RI','MA','VT','NH','ME']
    noncontiguous = ['AK','HI']
    north_canada = ['YT','NU','NT']
    british_columbia = ['BC']
    prarie_provinces = ['AB','SK','MB']
    ontario = ['ON']
    quebec = ['QC']
    atlantic_provinces = ['NL','PE','NB','NS']
    
    try:
        parsed = citystate.split("!")
        if(parsed[1] in pacific):
            return "pacific"
        elif(parsed[1] in rockies):
            return "rockies"
        elif(parsed[1] in southwest):
            return "southwest"
        elif(parsed[1] in midwest):
            return "midwest"
        elif(parsed[1] in southeast):
            return "southeast"
        elif(parsed[1] in northeast):
            return "northeast"
        elif(parsed[1] in noncontiguous):
            return "noncontiguous"
        elif(parsed[1] in north_canada):
            return "north_canada"
        elif(parsed[1] in british_columbia):
            return "british_columbia"
        elif(parsed[1] in prarie_provinces):
            return "prarie_provinces"
        elif(parsed[1] in ontario):
            return "ontario"
        elif(parsed[1] in quebec):
            return "quebec"
        elif(parsed[1] in atlantic_provinces):
            return "atlantic_provinces"
        else:
            if("UK/England" in parsed[0]):
                return "UK"
            else:
                return "other"
    except:
        return "other"
        
reports['region'] = reports['citystate'].apply(lambda x: region_separate(x))

/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Quinton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [111]:
# Grouping similar shapes
reports_shpgroup = reports.replace({'shape' : {'disk': 'circle','unknown':'other','teardrop':'oval',
        'egg': 'oval','fireball': 'sphere', 'delta':'triangle', 'pyramid':'triangle', 'cigar':'cylinder',
       'round': 'sphere', 'changed':'changing', 'flare':'light'}})

In [114]:
# One-hot encode our categorical variables.

cat_cols = ['city', 'state', 'shape', 'region']

for col in cat_cols:
    temp = pd.get_dummies(reports_shpgroup[col], prefix=col)
    reports_shpgroup = pd.concat([reports_shpgroup, temp], axis=1)

In [115]:
# Prep for tokenization/vectorization
texts=[" ".join(text) for text in reports_shpgroup['nostop_text'].values]

In [116]:
# Text vectorization at the word level 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
 
#word level tf-idf
Tfidf_vect = TfidfVectorizer(max_features=6000)

Tfidf_vect.fit(texts)
tfidf_vals = Tfidf_vect.transform(texts)

In [117]:
# Convert sparse matrix vectors to dataframe and join original dataframe
vectors_df = pd.DataFrame.sparse.from_spmatrix(tfidf_vals)

In [118]:
reports_wvectors = pd.concat([reports_shpgroup, vectors_df], axis = 1)

In [122]:
reports_shpgroup.columns[0:40]

Index(['summary', 'city', 'state', 'date_time', 'shape', 'duration', 'stats',
       'report_link', 'text', 'posted', 'city_latitude', 'city_longitude',
       'clean_text', 'tokenized_text', 'nostop_text', 'year', 'month', 'day',
       'hour', 'minute', 'duration_parsed', 'citystate', 'region',
       'city_((HOAX))', 'city_((Location unspecified)) (UK/England)',
       'city_((Unspecified by witness))', 'city_((Unspecified))',
       'city_((unspecified location))', 'city_(Norway)',
       'city_(Republic of Ireland)', 'city_1-25 corridor', 'city_100 Mile',
       'city_100 Mile House', 'city_14 Island Lake',
       'city_22degrees01 north  55degrees40 north, none', 'city_29 Palms',
       'city_70 Mile House', 'city_?', 'city_??', 'city_??? (Australia)'],
      dtype='object')

In [138]:
# Drop deprecated or unnecessary columns
reports_final = reports_wvectors.drop(columns = ['summary', 'city', 'state', 'date_time', 'shape', 'duration', 'stats',
                                                   'report_link', 'text', 'posted', 'region',
                                                   'clean_text', 'tokenized_text', 'nostop_text', 'citystate'])

In [ ]:
reports_final.to_csv(r'reports_with_vectors.csv')

# Predicting Shape from Text: Naive Bayes

In [53]:
shape_df = reports[pd.notnull(reports['shape'])]

In [54]:
shape_df.head()

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude,clean_text,tokenized_text,nostop_text
0,"Three saucer shaped ships. High in the sky, m...",Salem,OR,NaN,disk,15 minutes,Occurred : 8/15/1950 14:00 (Entered as : 08/1...,http://www.nuforc.org/webreports/135/S135871.html,"Three saucer shaped ships. High in the sky, m...",NaN,44.941247,-123.004235,Three saucer shaped ships High in the sky met...,"[saucer, shaped, ships, High, in, the, sky, me...","[saucer, shaped, ships, High, sky, metallic, s..."
1,Tear-drop shaped silent craft with sectional f...,Ellsworth,ME,NaN,teardrop,5 minutes,Occurred : 8/15/1967 21:30 (Entered as : 08/0...,http://www.nuforc.org/webreports/132/S132451.html,Tear-drop shaped silent craft with sectional f...,NaN,44.651300,-68.450700,Teardrop shaped silent craft with sectional fr...,"[shaped, silent, craft, with, sectional, front...","[shaped, silent, craft, sectional, front, end,..."
2,Green orb shot out of white light ((NUFORC No...,San Antonio,TX,2017-01-25T20:45:00,light,90 minutes,Occurred : 1/25/2017 20:45 (Entered as : 01/2...,http://www.nuforc.org/webreports/132/S132305.html,Green orb shot out of white light I saw a whit...,2017-01-26T00:00:00,29.488866,-98.475240,Green orb shot out of white light I saw a whit...,"[orb, shot, out, of, white, light, I, saw, a, ...","[orb, shot, white, light, I, saw, white, light..."
3,Saw three circular orange lights traveling sou...,Port Saint Lucie,FL,2017-02-24T20:45:00,circle,5 minutes,Occurred : 2/24/2017 20:45 (Entered as : 02/2...,http://www.nuforc.org/webreports/132/S132890.html,Saw three circular orange lights traveling sou...,2017-03-10T00:00:00,27.285686,-80.363444,Saw three circular orange lights traveling sou...,"[three, circular, orange, lights, traveling, s...","[three, circular, orange, lights, traveling, s..."
4,Light moving slowly thru the sky towards the W...,Whitefish,MT,2017-02-22T21:00:00,light,15 minutes,Occurred : 2/22/2017 21:00 (Entered as : 02/2...,http://www.nuforc.org/webreports/132/S132774.html,Light moving slowly thru the sky towards the w...,2017-03-10T00:00:00,48.411100,-114.337600,Light moving slowly thru the sky towards the w...,"[moving, slowly, thru, the, sky, towards, the,...","[moving, slowly, thru, sky, towards, west, Fir..."


In [55]:
shape_df.isnull().sum()

summary               8
city                141
state              7973
date_time          2262
shape                 0
duration           2757
stats                52
report_link           0
text                  0
posted             2262
city_latitude     21769
city_longitude    21769
clean_text            0
tokenized_text        0
nostop_text           0
dtype: int64

In [56]:
shape_df['shape'].value_counts()

light        22761
circle       11499
triangle     10488
fireball      8652
unknown       7879
other         7724
sphere        7362
disk          7005
oval          5033
formation     3456
changing      2715
cigar         2679
flash         1963
rectangle     1848
cylinder      1725
diamond       1650
chevron       1294
teardrop      1013
egg           1004
cone           468
cross          344
delta            8
crescent         2
round            2
dome             1
changed          1
pyramid          1
hexagon          1
flare            1
Name: shape, dtype: int64

In [57]:
# Grouping similar shapes
new_shape_df = shape_df.replace({'shape' : {'disk': 'circle','unknown':'other','teardrop':'oval',
        'egg': 'oval','fireball': 'sphere', 'delta':'triangle', 'pyramid':'triangle', 'cigar':'cylinder',
       'round': 'sphere', 'changed':'changing', 'flare':'light'}})

In [58]:
new_shape_df['shape'].value_counts()

light        22762
circle       18504
sphere       16016
other        15603
triangle     10497
oval          7050
cylinder      4404
formation     3456
changing      2716
flash         1963
rectangle     1848
diamond       1650
chevron       1294
cone           468
cross          344
crescent         2
dome             1
hexagon          1
Name: shape, dtype: int64

In [59]:
texts=[" ".join(text) for text in new_shape_df['nostop_text'].values]

In [60]:
Encoder = LabelEncoder()
y= Encoder.fit_transform(new_shape_df['shape'])

In [61]:
num_classes = len(np.unique(y))

In [62]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(texts, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=1)

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#TEXT REPRESENTATION
#word level tf-idf
Tfidf_vect = TfidfVectorizer(max_features=6000)

Tfidf_vect.fit(texts)
x_train_tfidf = Tfidf_vect.transform(x_train)
x_test_tfidf = Tfidf_vect.transform(x_test)


# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(texts)
X_train_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(texts)
X_train_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_train) 
X_test_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_test) 


from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=4)
X_counts = count_vect.fit(texts)
x_train_count =  count_vect.transform(x_train)
x_test_count =  count_vect.transform(x_test)

KeyboardInterrupt: 

In [ ]:
x_train_tfidf

In [ ]:
tfidf_sparse_df = pd.DataFrame.sparse.from_spmatrix(x_train_tfidf)

In [ ]:
# fit the training dataset on the NB classifier
from sklearn.naive_bayes import MultinomialNB

# Naive Bayes on Count Vectors
def NaiveBayes(X_train, Y_train, X_test, Y_test, vectorizer):
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(X_train,Y_train)
    predictions_NB = Naive.predict(X_test)
    probs = Naive.predict_proba(X_test)
    probability = np.amax(probs, 1)
    print(vectorizer,"Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)
    print(metrics.classification_report(Y_test, predictions_NB))
    summary = pd.DataFrame({'%':probability,'Prediction':predictions_NB})
    print(summary)


NaiveBayes(x_train_count, y_train, x_test_count, y_test, 'Count')
NaiveBayes(x_train_tfidf, y_train, x_test_tfidf, y_test, 'Word')
NaiveBayes(X_train_tfidf_ngram, Y_train, X_test_tfidf_ngram, Y_test, 'Ngram')
NaiveBayes(X_train_tfidf_ngram_chars, Y_train, X_test_tfidf_ngram_chars, Y_test, 'Ngram Char')


